NAS implementation


Genetic Algorithm for find the best neural architecture

Create population

In [ ]:
import random
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [ ]:
type_feature_extraction = "feature extraction"
hidden_layer = "hidden layer"
type_activation_function = "activation function"
fitness = 'fitness'

num_of_generations = 10
num_of_chromosomes = 10

crossover_rate = 0.8 # select parents for crossover with this rate
mutation_rate = 0.01

In [ ]:
def create_chromosomes(num_chromosomes):
    chromosomes = []
    for i in range(num_chromosomes):
        chro = dict()
        chro[type_feature_extraction] = random.randint(1,3)
        num_hidden_layer = random.randint(0, 2)
        chro[hidden_layer] = [random.choice([10, 20, 30]) for _ in range(num_hidden_layer)]
        chro[type_activation_function] = [random.choice([1, 2]) for _ in range(num_hidden_layer + 1)]
        chro[fitness] = 0.0
        chromosomes.append(chro)

    return chromosomes

In [ ]:
def evaluate_fitness_population(population):
    pass

In [ ]:
def select_parents(population):
    num_selected = int(crossover_rate*len(population))
    return population[:num_selected]

In [ ]:
def crossover_feature(parent1, parent2):
    parent1 , parent2 = parent2 , parent1
    return parent1 , parent2

In [ ]:
def crossover_layer_activation(parent1_layer, parent2_layer, parent1_active, parent2_active):
    parent1_len = len(parent1_layer)
    parent2_len = len(parent2_layer)
    
    if parent1_len - parent2_len == -1:
        value = parent2_layer.pop()
        parent1_layer.append(value)
        value = parent2_active.pop()
        parent1_active.append(value)

    elif parent1_len - parent2_len == 1:
        value = parent1_layer.pop()
        parent2_layer.append(value)
        value = parent1_active.pop()
        parent2_active.append(value)

    elif parent1_len == parent2_len and parent1_len != 0:
        parent1_layer[-1], parent2_layer[-1] = parent2_layer[-1], parent1_layer[-1]
        parent1_active[-1], parent2_active[-1] = parent2_active[-1], parent1_active[-1]

    else:
        if parent2_len >parent1_len:
            value = parent2_layer.pop()
            parent1_layer.append(value)
            value = parent2_active.pop()
            parent1_active.append(value)
        else:
            value = parent1_layer.pop()
            parent2_layer.append(value)
            value = parent1_active.pop()
            parent2_active.append(value)
    
    
    return parent1_layer, parent2_layer, parent1_active, parent2_active

In [ ]:
def crossover(parents):
    offspring = list()
    for i in range(int(len(parents)/2)):
        child1 = dict()
        child2 = dict()

        child1[type_feature_extraction], child2[type_feature_extraction] = crossover_feature(parents[2*i][type_feature_extraction], parents[2*i+1][type_feature_extraction])
        child1[hidden_layer], child2[hidden_layer], child1[type_activation_function], child2[type_activation_function] = crossover_layer_activation(parents[2*i][hidden_layer], parents[2*i+1][hidden_layer],parents[2*i][type_activation_function], parents[2*i+1][type_activation_function])

        child1[fitness] = 0
        child2[fitness] = 0
        
        offspring.append(child1)
        offspring.append(child2)
    
    return offspring
        

In [ ]:
def mutation_feature(parent):
    parent = random.randint(1,3)
    return parent

In [ ]:
def mutation_activition(parent):
    index = random.randint(0, len(parent)-1)
    parent[index] = random.randint(1,2)
    return parent

In [ ]:
def mutation_layer(parent):
    index = random.randint(0, len(parent)-1)
    parent[index] = random.choice([10, 20, 30])
    return parent

In [ ]:
def mutate(offspring):
    for child in offspring:
        if random.uniform(0, 1) <= mutation_rate:
            child[type_feature_extraction] = mutation_feature(child[type_feature_extraction])
        if random.uniform(0, 1) <= mutation_rate:
            child[type_activation_function] = mutation_activition(child[type_activation_function])
        if random.uniform(0, 1) <= mutation_rate:
            child[hidden_layer] = mutation_layer(child[hidden_layer])
    
    return offspring

In [ ]:
def select_survivors(population, offspring):
    while(len(offspring) > 0):
        best_child = max(offspring, key=lambda x: x[fitness])
        bad_parent = min(population, key=lambda x: x[fitness])
        if best_child[fitness] > bad_parent[fitness]:
            population.remove(bad_parent)
            population.append(best_child)
            offspring.remove(best_child)
        else:
            break
    
    return population

In [ ]:
def genetic_algorithm():
    # Initialize population
    population = create_chromosomes(num_of_chromosomes)

    # Evaluate population fitness
    # fitness = accuracy of the model 
    evaluate_fitness_population(population)

    # Sort population
    population.sort(key = lambda x: x[fitness])

    # While termination condition is not met
    for _ in range(num_of_generations):
        # Select parents
        parents = select_parents(population)

        # Apply crossover / mutation operators
        offspring = crossover(parents)
        mutate(offspring)

        # Evaluate offspring
        evaluate_fitness_population(offspring)

        # Select survivors
        population = select_survivors(population, offspring)

        # Sort population
        population.sort(key = lambda x: x[fitness])

    return max(population, key=lambda x: x[fitness])   # Return best individual in population